In [1]:
import pandas as pd
import numpy as np
import gzip
import os
import glob

## Data cleaning

### Looping over every zipped state data and read as csv

In [4]:

zipfiles = "/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_shipment/*.gz"

filelist = glob.glob(zipfiles)

select_data = []

for gzfile in filelist:
    raw = pd.read_csv(gzfile, iterator= True, chunksize = 10000, compression='gzip', error_bad_lines=False)


    for df in raw:

        df["TRANSACTION_DATE1"] = pd.to_datetime(df["TRANSACTION_DATE"], format='%m%d%Y')

        df["year"] = pd.DatetimeIndex(df["TRANSACTION_DATE1"]).year

        df = df[["BUYER_COUNTY", "BUYER_STATE", "year", "MME"]]

        select_data.append(df)



whole_data = pd.concat(select_data)

grouped = whole_data.groupby(["BUYER_STATE", "BUYER_COUNTY", "year"], as_index=False).sum("MME")



# Test if the data is correctly loaded

print(len(grouped))

grouped["BUYER_STATE"].value_counts()

/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


5984


GA    1366
MS     729
OK     693
AL     603
FL     600
LA     571
MT     430
SC     414
OR     312
NM     266
Name: BUYER_STATE, dtype: int64

In [5]:
grouped[(grouped["year"]==2006)& (grouped["BUYER_COUNTY"]=="BERNALILLO")]

,BUYER_STATE,BUYER_COUNTY,year,MME
4299,NM,BERNALILLO,2006,1.915714e+08


### Store data in parquet form

In [6]:
grouped.to_parquet('/Users/yangshining/pds2021-opioids-pds6/10_modified_data/opioid_whole_final', engine='fastparquet')

In [3]:
# Test stuff (Don't run!!!!!!)
test = pd.read_parquet("/Users/yangshining/pds2021-opioids-pds6/10_modified_data/opioid_whole.parquet", engine='fastparquet')

test[(test["year"]==2006)& (test["BUYER_COUNTY"]=="BERNALILLO")]


,BUYER_COUNTY,BUYER_STATE,year,MME
index,,,,
0,BERNALILLO,NM,2006,8.945369e+05
0,BERNALILLO,NM,2006,1.174579e+06
0,BERNALILLO,NM,2006,4.902307e+05
0,BERNALILLO,NM,2006,1.951027e+06
0,BERNALILLO,NM,2006,2.401950e+06
...,...,...,...,...
0,BERNALILLO,NM,2006,2.483118e+06
0,BERNALILLO,NM,2006,1.227908e+06
0,BERNALILLO,NM,2006,1.108440e+06
